In [2]:
from synthetic_data import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from get_real_data import *

import tensorflow as tf
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, Conv1D, Flatten, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers

2023-09-01 17:22:28.817815: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-01 17:22:28.819276: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-01 17:22:28.847182: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-01 17:22:28.847921: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-01 17:22:29.411243: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
X_ris_wedg, y_ris_wedg, X_fal_wedg, y_fal_wedg, X_d_top, y_d_top, X_d_bottom, y_d_bottom = get_X_y(noise=False, general=True)

In [3]:
X_rw_noise, y_rw_noise, X_fw_noise, y_fw_noise, X_dt_noise, y_dt_noise, X_db_noise, y_db_noise = get_hack_noise(X_ris_wedg, y_ris_wedg, X_fal_wedg, y_fal_wedg, X_d_top, y_d_top, X_d_bottom, y_d_bottom, synth=True, real=True, f=True)

In [4]:
X_synth = X_ris_wedg + X_rw_noise + X_fal_wedg + X_fw_noise + X_d_top + X_dt_noise + X_d_bottom + X_db_noise
y_synth = y_ris_wedg + y_rw_noise + y_fal_wedg + y_fw_noise + y_d_top + y_dt_noise + y_d_bottom + y_db_noise

In [3]:
X1, y1, X2, y2, X3, y3, X4, y4 = get_real_X_y()

In [5]:
X1_noise, y1_noise, X2_noise, y2_noise, X3_noise, y3_noise, X4_noise, y4_noise = get_hack_noise(X1, y1, X2, y2, X3, y3, X4, y4, synth=False, real=True, f=False)

In [4]:
X_up, y_up, X_down, y_down = get_up_down(patterns=["uptrend","downtrend"])

In [6]:
X_real = X1 + X1_noise + X2 + X2_noise + X3 + X3_noise + X4 + X4_noise + X_up + X_down
y_real = y1 + y1_noise + y2 + y2_noise + y3 + y3_noise + y4 + y4_noise + y_up + y_down


In [83]:
X = X_real + X_synth
y = y_real + y_synth

In [10]:
def onehotencode(y_synth):
    y_synth = np.array(y_synth)
    start = y_synth[:,0]
    end = y_synth[:,1]
    pattern = y_synth[:,2]
    mapping = {
        0: [0, 0, 0, 0],
        1: [0, 0, 0, 1],
        2: [0, 0, 1, 0],
        3: [0, 1, 0, 0],
        4: [1, 0, 0, 0]
    }
    mapped_values = [tuple(mapping[val]) for val in pattern]
    y_s = list(np.column_stack((start, end, mapped_values)))
    return y_s


In [11]:
y_preprocessed = onehotencode(y)

In [84]:
X_preprocessed = pad_sequences(X, dtype='float32', padding='post', value=-1)


In [65]:
l = X_train[:,:,0]
z = X_train[:,:,1]

In [88]:
l.size

2446748

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y_preprocessed, test_size=0.2)

In [86]:
open_scaler = StandardScaler()
X_o_train = open_scaler.fit_transform(X_train[:,:,0])
X_o_test = open_scaler.transform(X_test[:,:,0])
high_scaler = StandardScaler()
X_h_train = high_scaler.fit_transform(X_train[:,:,1])
X_h_test = high_scaler.transform(X_test[:,:,1])
low_scaler = StandardScaler()
X_l_train = low_scaler.fit_transform(X_train[:,:,2])
X_l_test = low_scaler.transform(X_test[:,:,2])
close_scaler = StandardScaler()
X_c_train = close_scaler.fit_transform(X_train[:,:,3])
X_c_test = close_scaler.transform(X_test[:,:,3])



In [2]:
X_train_preprocessed = []
X_test_preprocessed = []
for i in range(len(X_o_train)):
    X_train_preprocessed.append(np.column_stack((X_o_train, X_h_train, X_l_train, X_c_train)))
    X_test_preprocessed.append(np.column_stack((X_o_test, X_h_test, X_l_test, X_c_test)))



NameError: name 'X_o_train' is not defined

### We have to change with the functional api, because is a two problem one for regresion and other for classification

### We have to add an intermediate layer

### Look into generative models

In [ ]:
X_train_preprocessed = tf.convert_to_tensor(X_train_preprocessed, np.float32)
y_train = tf.convert_to_tensor(y_train, np.int16)

display(type(X_train_preprocessed))
display(type(y_train))

In [ ]:
input_shape = X_train_preprocessed.shape[1:]

def initialize_model_CNN():
    model = Sequential()

    model.add(Masking(mask_value=-1, input_shape=input_shape))
    model.add(Conv1D(32, activation='relu', kernel_size=3, kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3))),
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(32, activation='relu', kernel_size=3))
    
    model.add(Flatten())
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=3, activation='linear'))

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

In [ ]:
es = EarlyStopping(patience = 5, restore_best_weights=True)

model = initialize_model_CNN()

model.fit(
    X_train_preprocessed,
    y_train,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)